In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
from sklearn import datasets
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge #ordinary linear regression + w/ ridge regularization
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import folium

In [32]:
df1 = pd.read_csv('central_park_weather.csv')

In [33]:
df1.columns

Index(['STATION', 'NAME', 'DATE', 'AWND', 'PRCP', 'SNOW', 'SNWD', 'TMAX',
       'TMIN'],
      dtype='object')

In [34]:
df1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4595 entries, 0 to 4594
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   STATION  4595 non-null   object 
 1   NAME     4595 non-null   object 
 2   DATE     4595 non-null   object 
 3   AWND     4370 non-null   float64
 4   PRCP     4595 non-null   float64
 5   SNOW     4594 non-null   float64
 6   SNWD     4595 non-null   float64
 7   TMAX     4595 non-null   int64  
 8   TMIN     4595 non-null   int64  
dtypes: float64(4), int64(2), object(3)
memory usage: 323.2+ KB


In [35]:
df1['DATE']=pd.to_datetime(df1['DATE'])

In [37]:
df1['DATEyear']=df1['DATE'].dt.year

In [39]:
df1['DATEmonth']= df1['DATE'].dt.month

In [40]:
df1['DATEday']= df1['DATE'].dt.day

In [41]:
df1

,STATION,NAME,DATE,AWND,PRCP,SNOW,SNWD,TMAX,TMIN,DATEyear,DATEmonth,DATEday
0,USW00094728,"NY CITY CENTRAL PARK, NY US",2009-01-01,11.18,0.00,0.0,0.0,26,15,2009,1,1
1,USW00094728,"NY CITY CENTRAL PARK, NY US",2009-01-02,6.26,0.00,0.0,0.0,34,23,2009,1,2
2,USW00094728,"NY CITY CENTRAL PARK, NY US",2009-01-03,10.07,0.00,0.0,0.0,38,29,2009,1,3
3,USW00094728,"NY CITY CENTRAL PARK, NY US",2009-01-04,7.61,0.00,0.0,0.0,42,25,2009,1,4
4,USW00094728,"NY CITY CENTRAL PARK, NY US",2009-01-05,6.93,0.00,0.0,0.0,43,38,2009,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...
4590,USW00094728,"NY CITY CENTRAL PARK, NY US",2021-07-27,2.91,0.00,0.0,0.0,89,73,2021,7,27
4591,USW00094728,"NY CITY CENTRAL PARK, NY US",2021-07-28,3.58,0.14,0.0,0.0,81,69,2021,7,28
4592,USW00094728,"NY CITY CENTRAL PARK, NY US",2021-07-29,3.80,0.47,0.0,0.0,77,69,2021,7,29
4593,USW00094728,"NY CITY CENTRAL PARK, NY US",2021-07-30,7.61,0.00,0.0,0.0,82,67,2021,7,30


In [44]:
df1 =df1[df1['DATEyear'] == 2020 ]

In [46]:
df1 =df1[df1['DATEmonth'] == 1 ]

In [49]:
df1.loc[0:,['DATEday']] = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]

C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [57]:
df1.drop('STATION',axis='columns', inplace=True)


C:\Users\user\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [64]:

df1.drop('DATE',axis='columns', inplace=True)
df1.drop('DATEyear',axis='columns', inplace=True)
df1.drop('DATEmonth',axis='columns', inplace=True)

In [65]:
df1

,AWND,PRCP,SNOW,SNWD,TMAX,TMIN,DATEday
4017,8.50,0.00,0.0,0.0,41,34,0
4018,5.37,0.00,0.0,0.0,49,33,1
4019,3.36,0.15,0.0,0.0,49,44,2
4020,4.47,0.27,0.0,0.0,51,41,3
4021,11.41,0.00,0.0,0.0,42,35,4
4022,6.04,0.04,0.2,0.0,45,32,5
4023,5.14,0.00,0.0,0.0,45,36,6
4024,9.62,0.00,0.0,0.0,42,28,7
4025,4.47,0.00,0.0,0.0,34,23,8
4026,6.26,0.00,0.0,0.0,55,33,9


In [68]:
df1.to_csv('weather20201.csv')

In [ ]:
df1.drop('NAME',axis='columns', inplace=True)